In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt

In [3]:
sales = pd.read_csv('../input/sales_train_v2.csv')

In [4]:
sales_g = sales.groupby(['date_block_num', 'shop_id', 'item_id'])['item_cnt_day'].sum().reset_index()

In [5]:
len(sales_g['item_id'].unique())

21807

In [ ]:
sales_g['item_id_item_cnt_day_sma_2'] = np.nan
sales_g['item_id_item_cnt_day_sma_5'] = np.nan
sales_g['item_id_item_cnt_day_sma_12'] = np.nan

sales_g['item_id_item_cnt_day_diff_1'] = np.nan
sales_g['item_id_item_cnt_day_diff_2'] = np.nan
sales_g['item_id_item_cnt_day_diff_5'] = np.nan

In [6]:
grouped_items = sales.groupby(['date_block_num', 'item_id'])['item_cnt_day'].sum().reset_index()
grouped_shops = sales.groupby(['date_block_num', 'shop_id'])['item_cnt_day'].sum().reset_index()

In [ ]:
i = 0
new_df=pd.DataFrame()
for item in grouped_items.item_id.unique():
    temp = grouped_items[grouped_items.item_id == item]
    temp.sort_values(by='date_block_num', inplace=True)

    for window in [2,5,12]:
        value = (temp.item_cnt_day - temp.item_cnt_day.rolling(window).mean()
                 ).fillna(0)
        temp['item_id_item_cnt_day_sma_{}'.format(window)] = value
    
    for window in [1, 2, 5, 12]:
        value = temp.item_cnt_day.diff(window).fillna(0)
        temp['item_id_item_cnt_day_diff_{}'.format(window)] = value
    new_df = new_df.append(temp)
    i+=1

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-ve

In [ ]:
sales_g['shop_id_item_cnt_day_sma_2'] = np.nan
sales_g['shop_id_item_cnt_day_sma_5'] = np.nan
sales_g['shop_id_item_cnt_day_sma_12'] = np.nan

sales_g['shop_id_item_cnt_day_diff_1'] = np.nan
sales_g['shop_id_item_cnt_day_diff_2'] = np.nan
sales_g['shop_id_item_cnt_day_diff_5'] = np.nan

In [ ]:
i = 0
new_df_2=pd.DataFrame()
for item in grouped_shops.shop_id.unique():
    temp = grouped_shops[grouped_shops.shop_id == item]
    temp.sort_values(by='date_block_num', inplace=True)

    for window in [2,5,12]:
        value = (temp.item_cnt_day - temp.item_cnt_day.rolling(window).mean()
                 ).fillna(0)
        temp['shop_id_item_cnt_day_sma_{}'.format(window)] = value
    
    for window in [1, 2, 5, 12]:
        value = temp.item_cnt_day.diff(window).fillna(0)
        temp['shop_id_item_cnt_day_diff_{}'.format(window)] = value
    new_df_2 = new_df_2.append(temp)
    i+=1

In [ ]:
cols_to_use_item = ['date_block_num', 'item_id']
for window in [2,5,12]:
    cols_to_use_item.append('item_id_item_cnt_day_sma_{}'.format(window))
for window in [1, 2, 5, 12]:
    cols_to_use_item.append('item_id_item_cnt_day_diff_{}'.format(window))    

In [ ]:
cols_to_use_shop = ['date_block_num', 'shop_id']
for window in [2,5,12]:
    cols_to_use_shop.append('shop_id_item_cnt_day_sma_{}'.format(window))
for window in [1, 2, 5, 12]:
    cols_to_use_shop.append('shop_id_item_cnt_day_diff_{}'.format(window))     


In [ ]:
new_df[cols_to_use_item].head()

In [ ]:
new_df[cols_to_use_item].to_csv('../item_sales_dynamics.csv', index=False)
new_df_2[cols_to_use_shop].to_csv('../shop_sales_dynamics.csv', index=False)

In [ ]:
final_df = pd.merge(new_df, new_df_2[cols_to_use], on = ['date_block_num', 'shop_id'], how='left')